In [2]:
# Import Dependenices
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif, f_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
%matplotlib inline
!pip install scikit-plot

In [3]:
# Load Dataset
df=pd.read_csv('D:\\injamul\\LabelingAFS.csv')
df.shape

(170366, 79)

In [4]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,1
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,1
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,1
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702162,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170361,55641,49,1,3,6,18,6,6,6.000000,0.000000,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1
170362,45337,217,2,1,31,6,31,0,15.500000,21.920310,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1
170363,22,1387547,41,46,2728,6634,456,0,66.536585,110.129945,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1
170364,22,207,1,1,0,0,0,0,0.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,1


In [5]:
df.isnull().sum().sort_values(ascending=False).head(100)

 Label                   0
 Bwd IAT Max             0
 Fwd IAT Mean            0
 Fwd IAT Std             0
 Fwd IAT Max             0
                        ..
 Avg Fwd Segment Size    0
 Avg Bwd Segment Size    0
 Fwd Header Length.1     0
Fwd Avg Bytes/Bulk       0
 Destination Port        0
Length: 79, dtype: int64

In [6]:
df=df.fillna(method='ffill')

In [7]:
y= df[' Label']
x= df.drop(' Label', axis=1)

In [8]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [9]:
nan_mask = np.isnan(x)
inf_mask = np.isinf(x)
has_nan = np.any(nan_mask)
has_inf = np.any(inf_mask)

In [10]:
x = x[~np.any(nan_mask, axis=1)]
x = x[~np.any(inf_mask, axis=1)]
y = y[~np.any(nan_mask, axis=1)]
y = y[~np.any(inf_mask, axis=1)]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


Recursive Feature Elimination (RFE)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [13]:
from sklearn.feature_selection import RFE
sel = RFE(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1), n_features_to_select = 15)
sel.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
    n_features_to_select=15)

In [14]:
sel.get_support()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True, False,  True,  True, False,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
        True,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False])

In [15]:
features = X_train.columns[sel.get_support()]
features

Index([' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Min', ' Fwd Header Length',
       'Fwd Packets/s', ' Bwd Packets/s', ' Max Packet Length',
       ' Packet Length Mean', ' Average Packet Size', ' Fwd Header Length.1',
       ' Init_Win_bytes_backward'],
      dtype='object')

In [16]:
len(features)

15

In [17]:
X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

In [23]:
def run_randomForest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))

In [20]:
%%time
run_randomForest(X_train_rfe, X_test_rfe, y_train, y_test)

Accuracy:  0.9952026630115528
Wall time: 12.4 s


In [21]:
%%time
run_randomForest(X_train, X_test, y_train, y_test)

Accuracy:  0.9957313491286469
Wall time: 14.2 s


In [21]:
%%time
run_randomForest(X_train_rfc, X_test_rfc, y_train, y_test)

Accuracy:  0.9957509300959467
Wall time: 12.7 s


Feature selection by GradientBoost Tree Importance

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
from sklearn.feature_selection import RFE

In [15]:
sel = RFE(GradientBoostingClassifier(n_estimators=100, random_state=0), n_features_to_select = 15)
sel.fit(X_train, y_train)

RFE(estimator=GradientBoostingClassifier(random_state=0),
    n_features_to_select=15)

In [16]:
sel.get_support()

array([False,  True, False, False,  True,  True, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False,  True, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False,  True,  True, False, False, False,
       False, False, False, False, False, False])

In [17]:
features = X_train.columns[sel.get_support()]

In [18]:
features

Index([' Flow Duration', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Mean',
       ' Bwd Packet Length Std', ' Flow IAT Min', ' Fwd IAT Std',
       ' Fwd IAT Min', ' Bwd Packets/s', ' Max Packet Length',
       ' Packet Length Mean', ' Avg Fwd Segment Size', ' Subflow Bwd Bytes',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd'],
      dtype='object')

In [19]:
len(features)

15

In [20]:
X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

In [24]:
%%time
run_randomForest(X_train_rfe, X_test_rfe, y_train, y_test)

Accuracy:  0.9954376346191501
Wall time: 2.02 s


In [25]:
%%time
run_randomForest(X_train, X_test, y_train, y_test)

Accuracy:  0.9957313491286469
Wall time: 3.3 s
